In [180]:

import numpy as np
import pandas as pd
import os


from datasetsforecast.hierarchical import HierarchicalData
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import  BottomUp, TopDown, MiddleOut, MinTrace, ERM
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive,ARIMA
from hierarchicalforecast.evaluation import HierarchicalEvaluation

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [12]:
# if u r running in colab uncomment the below
# from google.colab import drive

# drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
xdat = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
xdat['date'] = pd.to_datetime(xdat['date'])
xdat.head(3)

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0.0,2012-01-31,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,glass,500ml,0.96,13280.0
1,1.0,2012-01-31,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,plastic,1.5lt,2.86,6727.0
2,2.0,2012-01-31,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,can,330ml,0.87,9848.0


In [14]:
xdat['city'].unique()

array(['Athens', 'Irakleion', 'Patra', 'Thessaloniki', 'Larisa', nan],
      dtype=object)

In [15]:
xdat['shop'].unique()

array(['shop_1', 'shop_2', 'shop_6', 'shop_4', 'shop_3', 'shop_5', nan],
      dtype=object)

In [23]:
xdat = xdat[~xdat['shop'].isna()]

In [24]:
xdat['shop'] = xdat['shop'].apply(lambda s: s.replace('_',''))

# NA
xdat['container'] = xdat['container'].fillna('unk')
xdat['capacity'] = xdat['capacity'].fillna('unk')

# sanity check
for f in ['city','shop', 'brand', 'container', 'capacity']:
    print(f)
    print(xdat[f].unique())

city
['Athens' 'Irakleion' 'Patra' 'Thessaloniki' 'Larisa']
shop
['shop1' 'shop2' 'shop6' 'shop4' 'shop3' 'shop5']
brand
['kinder-cola' 'adult-cola' 'orange-power' 'gazoza' 'lemon-boost']
container
['glass' 'plastic' 'can' 'unk']
capacity
['500ml' '1.5lt' '330ml' 'unk']


In [26]:
xdat['shop'] = xdat['city'] + '_' + xdat['shop']
xdat['brand'] = xdat['shop'] + '_' + xdat['brand']
xdat['container'] = xdat['brand'] + '_' + xdat['container']
xdat['capacity'] = xdat['container'] + '_' + xdat['capacity']
xdat.head(3)

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0.0,2012-01-31,Athens,37.97945,23.71622,672130.0,Athens_shop1,Athens_shop1_kinder-cola,Athens_shop1_kinder-cola_glass,Athens_shop1_kinder-cola_glass_500ml,0.96,13280.0
1,1.0,2012-01-31,Athens,37.97945,23.71622,672130.0,Athens_shop1,Athens_shop1_kinder-cola,Athens_shop1_kinder-cola_plastic,Athens_shop1_kinder-cola_plastic_1.5lt,2.86,6727.0
2,2.0,2012-01-31,Athens,37.97945,23.71622,672130.0,Athens_shop1,Athens_shop1_kinder-cola,Athens_shop1_kinder-cola_can,Athens_shop1_kinder-cola_can_330ml,0.87,9848.0


In [31]:
xdat['cnt'] = xdat.groupby('capacity')['quantity'].transform('count')
print(xdat.shape)
xdat = xdat.loc[xdat['cnt'] == 72]
print(xdat.shape)

(6480, 13)
(4536, 13)


In [37]:
xdat.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity,cnt
0,0.0,2012-01-31,Athens,37.97945,23.71622,672130.0,Athens_shop1,Athens_shop1_kinder-cola,Athens_shop1_kinder-cola_glass,Athens_shop1_kinder-cola_glass_500ml,0.96,13280.0,72
3,3.0,2012-01-31,Athens,37.97945,23.71622,672130.0,Athens_shop1,Athens_shop1_adult-cola,Athens_shop1_adult-cola_glass,Athens_shop1_adult-cola_glass_500ml,1.00,20050.0,72
5,5.0,2012-01-31,Athens,37.97945,23.71622,672130.0,Athens_shop1,Athens_shop1_orange-power,Athens_shop1_orange-power_glass,Athens_shop1_orange-power_glass_500ml,1.00,15041.0,72
8,8.0,2012-01-31,Athens,37.97945,23.71622,672130.0,Athens_shop1,Athens_shop1_lemon-boost,Athens_shop1_lemon-boost_glass,Athens_shop1_lemon-boost_glass_500ml,0.70,18623.0,72
9,9.0,2012-01-31,Athens,37.97945,23.71622,672130.0,Athens_shop1,Athens_shop1_lemon-boost,Athens_shop1_lemon-boost_plastic,Athens_shop1_lemon-boost_plastic_1.5lt,2.21,9645.0,72


In [49]:
# capacity
df_cap = xdat.groupby(['date', 'capacity'])[['quantity']].sum()
df_cap.reset_index(inplace=True)
df_cap = df_cap.T.reset_index(drop=True).T
df_cap.columns = ['ds', 'unique_id', 'sales']

# container
df_cont = xdat.groupby(['date', 'container'])[['quantity']].sum()
df_cont.reset_index(inplace=True)
df_cont = df_cont.T.reset_index(drop=True).T
df_cont.columns = ['ds', 'unique_id', 'sales']

# brand
df_brand = xdat.groupby(['date', 'brand'])[['quantity']].sum()
df_brand.reset_index(inplace=True)
df_brand = df_brand.T.reset_index(drop=True).T
df_brand.columns = ['ds', 'unique_id', 'sales']

# store
df_store = xdat.groupby(['date', 'shop'])[['quantity']].sum()
df_store.reset_index(inplace=True)
df_store = df_store.T.reset_index(drop=True).T
df_store.columns = ['ds', 'unique_id', 'sales']

# city
df_city = xdat.groupby(['date', 'city'])[['quantity']].sum()
df_city.reset_index(inplace=True)
df_city = df_city.T.reset_index(drop=True).T
df_city.columns = ['ds', 'unique_id', 'sales']

# total
df_tot = xdat.groupby(['date'])[['quantity']].sum()
df_tot.reset_index(inplace=True)
df_tot['unique_id'] = 'Total'
df_tot.columns = ['ds', 'sales', 'unique_id' ]



# aggregate
dfx = pd.concat([df_cap, df_cont, df_brand, df_store, df_city, df_tot ], axis = 0)

# format
dfx.columns = ['ds','unique_id', 'y']
dfx['ds'] = pd.to_datetime(dfx['ds'])
dfx.head(5)


,ds,unique_id,y
0,2012-01-31,Athens_shop1_adult-cola_glass_500ml,20050.0
1,2012-01-31,Athens_shop1_adult-cola_plastic_1.5lt,12229.0
2,2012-01-31,Athens_shop1_gazoza_can_330ml,16367.0
3,2012-01-31,Athens_shop1_gazoza_plastic_1.5lt,26884.0
4,2012-01-31,Athens_shop1_kinder-cola_glass_500ml,13280.0


In [51]:
xset = set(dfx.unique_id)
list1 = list(xset)
list1.sort()

list2 = [f for f in list1 if f.count('_') == 4]
list2.sort()

S = np.zeros((len(list1), len(list2)))
S = pd.DataFrame(S); S.index = list1; S.columns = list2

# populate the summing matrix
S.loc['Total'] = 1
for xcol in list1[:-1]:
    fl = [f for f in list2 if xcol in f]
    S.loc[xcol][fl] = 1

In [65]:
# encode the tags (structure per hierarchy level)
tags = {}
tags['Country'] = df_tot.unique_id.unique()
tags['Country/City'] = df_city.unique_id.unique()
tags['Country/City/Store'] = df_store.unique_id.unique()
tags['Country/City/Store/Brand'] = df_brand.unique_id.unique()
tags['Country/City/Store/Brand/Cont'] = df_cont.unique_id.unique()
tags['Country/City/Store/Brand/Cont/Cap'] = df_cap.unique_id.unique()

In [67]:
# train / test split: last 6 months

horizon = 6

x_test = dfx.groupby('unique_id').tail(horizon)
x_train = dfx.drop(x_test.index)
x_test = x_test.set_index('unique_id')
x_train = x_train.set_index('unique_id')

In [ ]:
!pip install hierarchicalforecast statsforecast datasetsforecast

In [72]:

fcst = StatsForecast(df = x_train, models=[AutoARIMA(season_length= 12)], freq='M', n_jobs=-1)
x_hat = fcst.forecast(h = horizon)

In [90]:
# Reconcile the base predictions
reconcilers = [
    BottomUp(),
    TopDown(method='forecast_proportions'),
    MiddleOut(middle_level='Country/City/Store/Brand', top_down_method='forecast_proportions'),
    MiddleOut(middle_level='Country/City/Store', top_down_method='forecast_proportions')
]

hrec = HierarchicalReconciliation(reconcilers=reconcilers)
# x_hat_rec = hrec.reconcile(x_hat, x_train, S, tags)
x_hat_rec = hrec.reconcile(x_hat, S, tags)

In [81]:
evaluator = HierarchicalEvaluation(evaluators=[mape])

evaluator.evaluate(Y_hat_df = x_hat_rec, Y_test_df = x_test,
                   tags=tags, benchmark='AutoARIMA')

,,AutoARIMA,AutoARIMA/BottomUp,AutoARIMA/TopDown_method-forecast_proportions,AutoARIMA/MiddleOut_middle_level-Country/City/Store/Brand_top_down_method-forecast_proportions,AutoARIMA/MiddleOut_middle_level-Country/City/Store_top_down_method-forecast_proportions
level,metric,,,,,
Overall,mean_absolute_percentage_error-scaled,1.0,0.959063,0.951125,0.950023,1.856634
Country,mean_absolute_percentage_error-scaled,1.0,1.802316,1.0,1.061309,9.759553
Country/City,mean_absolute_percentage_error-scaled,1.0,1.080281,0.991751,0.905805,4.844007
Country/City/Store,mean_absolute_percentage_error-scaled,1.0,0.239547,0.242239,0.215671,1.0
Country/City/Store/Brand,mean_absolute_percentage_error-scaled,1.0,0.99395,1.004097,1.0,2.172292
Country/City/Store/Brand/Cont,mean_absolute_percentage_error-scaled,1.0,1.0,0.99003,0.992117,1.816549
Country/City/Store/Brand/Cont/Cap,mean_absolute_percentage_error-scaled,1.0,1.0,0.99003,0.992117,1.816549


In [183]:
x_test1 = x_test.copy()

In [184]:
x_test1.reset_index(inplace=True)

In [187]:
x_hat_rec1 = x_hat_rec.copy()

In [188]:
x_hat_rec1 = x_hat_rec1.reset_index()

In [191]:
merged_df = pd.merge(x_hat_rec1[['unique_id','ds','AutoARIMA']],x_test1,on=['unique_id','ds'],how='inner')

In [192]:
merged_df.columns = ['unique_id','ds','pred_reconciled','pred_orig']

In [194]:
merged_df = merged_df.set_index('unique_id')

In [195]:
def calculate_mape(actual_values, forecasted_values):

    n = len(actual_values)
    mape = (1 / n) * sum(abs((actual - forecasted) / actual) for actual, forecasted in zip(actual_values, forecasted_values)) * 100
    return mape

In [199]:
mse1 = calculate_mape(merged_df['pred_orig'].values, merged_df['pred_reconciled'].values)
print('overall rmse: ' + str(mse1))
for k in tags.keys():
    mse2 = calculate_mape(merged_df.loc[tags[k]]['pred_orig'].values, merged_df.loc[tags[k]]['pred_reconciled'])
    print(f"{k} hirerchy level: {mse2}")
    # print(k + ' rmse: ' + str(mse1) + ' -> ' + str(mse2) )

overall rmse: 26.491622885675792
Country hirerchy level: 4.0390320077513095
Country/City hirerchy level: 8.215150790406572
Country/City/Store hirerchy level: 40.312267693357576
Country/City/Store/Brand hirerchy level: 21.17048866169639
Country/City/Store/Brand/Cont hirerchy level: 27.961654433323286
Country/City/Store/Brand/Cont/Cap hirerchy level: 27.961654433323286
